# 数据分析与可视化

导入包

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

读入数据并复制一份进行预处理，方便绘图

In [3]:
data = pd.read_csv('data/ccf_offline_stage1_train.csv')
offline = data.copy()
offline.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [ ]:
offline['Distance'].fillna(-1,inplace=True)
offline['date_received'] = pd.to_datetime(offline['Date_received'], format='%Y%m%d')
offline['date'] = pd.to_datetime(offline['Date'], format='%Y%m%d')
offline['discount_rate'] = offline['Discount_rate'].map(lambda x:float(x) if ':' not in str(x) else 
                            (float(str(x).split(':')[0])-float(str(x).split(':')[1])) / float(str(x).split(':')[0])
offline['isManjian'] = offline['Discount_rate'].map(lambda x: 1 if ':' in str(x) else 0)
offline['weekday_Receive'] = offline['date_received'].apply(lambda x: x.isoweekday())
offline['label'] = list